In [1]:
import dask.dataframe as ddf
import pandas as pd
import datetime
import functools
import seaborn as sns
import matplotlib.pyplot as plt

### uploade the population data

In [2]:
pop = ddf.read_csv('Population/population_1969-2020.csv', dtype = {'fips': str, 'year': str})
pop = pop.compute()
pop = pop.loc[:, ~pop.columns.str.contains('^Unnamed')]
pop.head()

,year,fips,pop
0,1969,01001,22289
1,1969,01003,53425
2,1969,01005,20120
3,1969,01007,13586
4,1969,01009,19544


In [3]:
pop.shape

(162334, 3)

In [4]:
previous = '2016'
current = '2017'

In [5]:
pop_previous = pop[(pop['year'] == previous)]
pop_previous.shape

(3142, 3)

In [6]:
pop_current = pop[(pop['year'] == current)]
pop_current.shape

(3142, 3)

In [7]:
pop = pop[(pop['year'] == previous) | (pop['year'] == current)]
pop.shape

(6284, 3)

##### this means 3089 counties have population data in 1970-1979; there are 3102, 3108, 
##### or 3131, 3135 counties have population data since 1980

#### Calculate the population in each month by using linear interpolation method

In [8]:
pop = pop.sort_values(['fips', 'year'])
pop.head()

,year,fips,pop
146624,2016,01001,54312
149766,2017,01001,49949
146625,2016,01003,186502
149767,2017,01003,200416
146626,2016,01005,23029


In [9]:
pop.tail()

,year,fips,pop
152905,2017,56041,19711
149764,2016,56043,7649
152906,2017,56043,7473
149765,2016,56045,6013
152907,2017,56045,6404


In [10]:
def findCommon(df1, df2):
    fips1 = list(df1['fips'])
    fips2 = list(df2['fips'])
    row_valid = []
    for item in fips2:
        if item in fips1:
            row_valid.append(item)
    return row_valid

In [11]:
row_valid = findCommon(pop_previous, pop_current)
print(len(row_valid))

3142


#### when current year has more counties than previous year, drop off the extra counties

In [12]:
pop = pop[pop['fips'].isin(row_valid)]
pop.shape

(6284, 3)

In [13]:
len(pop['fips'].unique())

3142

In [14]:
import numpy as np

In [15]:
pop.index = np.arange(1, len(pop) + 1)

In [16]:
pop.head()

,year,fips,pop
1,2016,01001,54312
2,2017,01001,49949
3,2016,01003,186502
4,2017,01003,200416
5,2016,01005,23029


In [17]:
pop = pop.reset_index()

In [18]:
pop.head()

,index,year,fips,pop
0,1,2016,01001,54312
1,2,2017,01001,49949
2,3,2016,01003,186502
3,4,2017,01003,200416
4,5,2016,01005,23029


In [19]:
def lableMonth(index):
    if index % 2 == 1:
        return 0
    else:
        return 12

In [20]:
pop['month'] = pop['index'].apply(lableMonth)
pop.head()

,index,year,fips,pop,month
0,1,2016,01001,54312,0
1,2,2017,01001,49949,12
2,3,2016,01003,186502,0
3,4,2017,01003,200416,12
4,5,2016,01005,23029,0


In [21]:
def divideDataframe_example(df):
    sliced_dfs = []
    for i in range(0, len(df), 2):
        df_slice = df.iloc[i:i+2]
        df_slice['index'] = df_slice['month']
        sliced_dfs.append(df_slice)
    for i, df_slice in enumerate(sliced_dfs[:3]):
        print(f"DataFrame Slice {i+1}:")
        print(df_slice)
        print()

In [22]:
divideDataframe_example(pop)

/global/homes/w/wenhuan/.conda/envs/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


DataFrame Slice 1:
   index  year   fips    pop  month
0      0  2016  01001  54312      0
1     12  2017  01001  49949     12

DataFrame Slice 2:
   index  year   fips     pop  month
2      0  2016  01003  186502      0
3     12  2017  01003  200416     12

DataFrame Slice 3:
   index  year   fips    pop  month
4      0  2016  01005  23029      0
5     12  2017  01005  24017     12



In [23]:
def divideDataframe(df):
    sliced_dfs = []
    for i in range(0, len(df), 2):
        df_slice = df.iloc[i:i+2]
        df_slice['index'] = df_slice['month']
        sliced_dfs.append(df_slice)
    return sliced_dfs

In [24]:
import pandas as pd
year1 = '1969'
year2 = '1970'
# Original DataFrame
data = {'index': [0, 12], 'year': [year1, year2], 'fips': ['01001', '01001'], 'population': [1000, 2200], 'month': [0, 12]}
df_slice = pd.DataFrame(data)

def interpolate_population(df_slice):
    new_data = {'index': [], 'year': [], 'fips': [], 'population': [], 'month': []}
    
    for i in range(len(df_slice) - 1):
        start_month = df_slice.loc[i, 'month']
        end_month = df_slice.loc[i+1, 'month']
        start_population = df_slice.loc[i, 'population']
        end_population = df_slice.loc[i+1, 'population']
        year = df_slice.loc[i, 'year']
        fips = df_slice.loc[i, 'fips']
        
        months_diff = end_month - start_month
        population_diff = end_population - start_population
        population_step = population_diff / months_diff

        for month_offset in range(months_diff + 1):
            new_data['index'].append(start_month + month_offset)
            new_data['year'] = [year1, year2, year2, year2, year2, year2, year2, year2, year2, year2, year2, year2, year2]
            new_data['fips'].append(fips)
            new_data['population'].append(int(start_population + month_offset * population_step))
            new_data['month'].append(start_month + month_offset)
    
    return pd.DataFrame(new_data)

interpolated_df = interpolate_population(df_slice)
print(interpolated_df)

    index  year   fips  population  month
0       0  1969  01001        1000      0
1       1  1970  01001        1100      1
2       2  1970  01001        1200      2
3       3  1970  01001        1300      3
4       4  1970  01001        1400      4
5       5  1970  01001        1500      5
6       6  1970  01001        1600      6
7       7  1970  01001        1700      7
8       8  1970  01001        1800      8
9       9  1970  01001        1900      9
10     10  1970  01001        2000     10
11     11  1970  01001        2100     11
12     12  1970  01001        2200     12


In [25]:
def linear_interpolation(slice):
    years = list(slice['year'])
    year1 = years[0]
    year2 = years[1]
    population = list(slice['pop'])
    fips = list(slice['fips'])
    data = {'index': [0, 12], 'year': [year1, year2], 'fips': fips, 'pop': population, 'month': [0, 12]}
    df_slice = pd.DataFrame(data)
    
    new_data = {'index': [], 'year': [], 'fips': [], 'pop': [], 'month': []}
    
    for i in range(len(df_slice) - 1):
        start_month = df_slice.loc[i, 'month']
        end_month = df_slice.loc[i+1, 'month']
        start_population = df_slice.loc[i, 'pop']
        end_population = df_slice.loc[i+1, 'pop']
        fips = df_slice.loc[i, 'fips']
        
        months_diff = end_month - start_month
        population_diff = end_population - start_population
        population_step = population_diff / months_diff
        
        for month_offset in range(months_diff + 1):
            new_data['index'].append(start_month + month_offset)
            new_data['year'] = [year1, year2, year2, year2, year2, year2, year2, year2, year2, year2, year2, year2, year2]
            new_data['fips'].append(fips)
            new_data['pop'].append(int(start_population + month_offset * population_step))
            new_data['month'].append(start_month + month_offset)
            
    return pd.DataFrame(new_data)

In [26]:
sliced_dfs = divideDataframe(pop)
new_dfs = []
for slice in sliced_dfs:
    new_slice = linear_interpolation(slice)
    new_dfs.append(new_slice)
    
df_new = pd.concat(new_dfs)
df_new.head()

/global/homes/w/wenhuan/.conda/envs/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


,index,year,fips,pop,month
0,0,2016,01001,54312,0
1,1,2017,01001,53948,1
2,2,2017,01001,53584,2
3,3,2017,01001,53221,3
4,4,2017,01001,52857,4


In [27]:
df_new

,index,year,fips,pop,month
0,0,2016,01001,54312,0
1,1,2017,01001,53948,1
2,2,2017,01001,53584,2
3,3,2017,01001,53221,3
4,4,2017,01001,52857,4
...,...,...,...,...,...
8,8,2017,56045,6273,8
9,9,2017,56045,6306,9
10,10,2017,56045,6338,10
11,11,2017,56045,6371,11


In [28]:
3135 + 3135*12

40755

In [29]:
df_monthly = df_new[df_new['year'] == current]
df_monthly.head()

,index,year,fips,pop,month
1,1,2017,01001,53948,1
2,2,2017,01001,53584,2
3,3,2017,01001,53221,3
4,4,2017,01001,52857,4
5,5,2017,01001,52494,5


In [30]:
replace_month = {1: 'Jan', 2: 'Feb', 3: 'Mar', 4: 'Apr', 5: 'May', 6: 'Jun', 7: 'Jul', 8: 'Aug', 9: 'Sep', 
                10: 'Oct', 11: 'Nov', 12: 'Dec'}
df_monthly['month'] = df_monthly['month'].replace(replace_month)
df_monthly.head()

/global/homes/w/wenhuan/.conda/envs/myenv/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,index,year,fips,pop,month
1,1,2017,01001,53948,Jan
2,2,2017,01001,53584,Feb
3,3,2017,01001,53221,Mar
4,4,2017,01001,52857,Apr
5,5,2017,01001,52494,May


In [31]:
df_monthly = df_monthly.drop('index', axis = 1)
df_monthly.head()

,year,fips,pop,month
1,2017,01001,53948,Jan
2,2017,01001,53584,Feb
3,2017,01001,53221,Mar
4,2017,01001,52857,Apr
5,2017,01001,52494,May


In [32]:
df_monthly.to_csv('Population/population_monthly/population_monthly_' + current + '.csv')